In [1]:
import os
import sys
import time
import json
import warnings
import numpy as np
import pandas as pd
import datetime as dtm

try:
    root = os.path.dirname(os.path.realpath(__file__))
except:
    root = os.getcwd()
    
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
    from afc.ctrlWrapper import Controller, make_inputs
    from afc.utility.weather import read_tmy3
    from afc.radiance.configs import get_config
    from afc.defaultConfig import default_parameter
    from afc.utility.plotting import plot_standard1
from doper.utility import download_cbc
    
# configure mpl
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 72
%matplotlib inline

In [2]:
# read weather (forecast) data
weather_path = os.path.join(os.path.dirname(root), 'dev', 'resources', 'weather', 
    'USA_CA_San.Francisco.Intl.AP.724940_TMY3.csv')
weather, info = read_tmy3(weather_path)
weather = weather.resample('5T').interpolate()
st = dtm.datetime(dtm.datetime.now().year, 7, 1)
wf = weather.loc[st:st+pd.DateOffset(hours=24),]
df = wf[['DryBulb','DNI','DHI','Wspd']].copy()
df = df[df.index.date == df.index[0].date()]

# Initialize controller
ctrl = Controller()

# Default parameter and inputs
parameter = default_parameter(precompute_radiance=False)
inputs = make_inputs(parameter, df)

# Query controller
ctrl.do_step(inputs=inputs) # Initialize
# print('Log-message:\n', ctrl.do_step(inputs=inputs))
# print('Duration:\n', ctrl.get_output(keys=['rad_duration','varts_duration','optall_duration','glare_duration',
#                                            'opt_duration','outputs_duration','duration']))
# print('Optimization:\n', ctrl.get_output(keys=['opt_objective','opt_duration','opt_termination','duration']))
# df = pd.DataFrame(ctrl.get_output(keys=['df_output'])['df_output'])
# df.index = pd.to_datetime(df.index, unit='ms')

'Duration [s]\t\t1.91\nObjective [$]\t\t19.43\t\t\t7.54 (Total Cost)\nCost [$]\t\t12.33 (Energy)\t6.98 (Demand)\nCO2 Emissions [kg]\t\t0.0\n'

In [3]:
parameter = ctrl.input['parameter']

res = pd.DataFrame()
for ver in ['2.10.3', '2.10.5', '2.10.6', '2.10.8', '2.10.10']:
    
    # specify cbc version
    if ver == '2.10.3':
        solver_path = '/home/Christoph/Documents/PrivateRepos/ESTCP-Controller/doper_private/src/DOPER/solvers/Linux64/cbc_original'
    elif ver == '2.10.5':
        solver_path = '/home/Christoph/Documents/PrivateRepos/ESTCP-Controller/doper_private/src/DOPER/solvers/Linux64/cbc'
    else:
        solver_path = os.path.join(root, f'cbc_{ver}')
        if not os.path.exists(solver_path):
            download_cbc(cbc_version=ver, root=root)
    
    # run optimization
    ctrl.controller.solver_path = solver_path
    t = ctrl.controller.do_optimization(ctrl.data, parameter=parameter,
                                        options=ctrl.input['parameter']['solver_options'])
    
    # save results
    duration, objective, df, _, _, termination, _ = t
    res.loc[ver, 'duration'] = duration
    res.loc[ver, 'objective'] = objective
    res.loc[ver, 'termination'] = termination
    res.loc[ver, 'df'] = df.to_json()

In [4]:
res

,duration,objective,termination,df
2.10.3,1.757971,19.428663,optimal,"{""Import Power [kW]"":{""1688169600000"":0.330632..."
2.10.5,5.465958,19.428663,optimal,"{""Import Power [kW]"":{""1688169600000"":0.330632..."
2.10.6,1.950843,19.428663,optimal,"{""Import Power [kW]"":{""1688169600000"":0.330632..."
2.10.8,1.925287,19.428663,optimal,"{""Import Power [kW]"":{""1688169600000"":0.330632..."
2.10.10,1.903002,19.428663,optimal,"{""Import Power [kW]"":{""1688169600000"":0.330632..."


In [5]:
# plot_standard1(pd.concat([wf, df], axis=1).ffill())

In [6]:
# for r in res.iterrows():
#     pd.read_json(r[1]['df'])['Import Power [kW]'].plot()